In [ ]:
# [Celda 1] - Introducción
# Single iteration example
"""
Una vez que estés satisfecho con la primera versión de tu funcionalidad, puedes construir 
un planificador alrededor de esto para ejecutar regularmente o escuchar eventos.
"""

In [32]:
# [Celda 2] - Setup del cliente Anthropic
# DIFERENCIA: Aquí usamos anthropic en lugar de openai
import os
from dotenv import load_dotenv
from anthropic import Anthropic  # Nueva importación
load_dotenv()

# DIFERENCIA: Configuración específica para Anthropic
anthropic_client = Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),  # Usa ANTHROPIC_API_KEY en lugar de OPENAI_API_KEY
)

# Si obtienes un error aquí, asegúrate de que:
# 1. Hayas instalado el paquete anthropic: pip install anthropic
# 2. Tengas un archivo .env con ANTHROPIC_API_KEY="tu_clave_api_aqui"

In [33]:
# [Celda 3] - Setup Hyperliquid SDK
# Esta parte no cambia respecto a las versiones anteriores
from hyperliquid.info import Info
from hyperliquid.utils import constants
import json

info = Info(constants.TESTNET_API_URL, skip_ws=True)

# Esta es una dirección de ejemplo
user_state = info.user_state("0xcd5051944f780a621ee62e39e493c489668acf4d") 
print(json.dumps(user_state, indent=2))

{
  "marginSummary": {
    "accountValue": "327149.602201",
    "totalNtlPos": "44260.82962",
    "totalRawUsd": "352872.449681",
    "totalMarginUsed": "2229.320572"
  },
  "crossMarginSummary": {
    "accountValue": "327149.602201",
    "totalNtlPos": "44260.82962",
    "totalRawUsd": "352872.449681",
    "totalMarginUsed": "2229.320572"
  },
  "crossMaintenanceMarginUsed": "442.608293",
  "withdrawable": "324920.281629",
  "assetPositions": [
    {
      "type": "oneWay",
      "position": {
        "coin": "ATOM",
        "szi": "491.45",
        "leverage": {
          "type": "cross",
          "value": 20
        },
        "entryPx": "10.9725",
        "positionValue": "3064.6822",
        "unrealizedPnl": "-2327.752925",
        "returnOnEquity": "-8.63340169",
        "liquidationPx": null,
        "marginUsed": "153.23411",
        "maxLeverage": 50,
        "cumFunding": {
          "allTime": "-1615.831717",
          "sinceOpen": "-1615.831717",
          "sinceChange": "

In [34]:
# [Celda 4] - Definición de identidad del usuario
# Esta parte no cambia
identity_map = {
    "risk_level": "Moderate risk, willing to risk some money for the right investments but not chasing every new opportunity.",
    "token_preferences": "Likes ETH more than BTC, doesnt like SOL",
    "mission_statement": "Accumulate as much ETH as possible given the available funds.",
}

In [35]:
# [Celda 5] - Recopilación de datos on-chain
# Esta parte no cambia
available_balance = user_state.get('withdrawable')
print(f"Available balance: {available_balance} USD")

positions = user_state.get('assetPositions')
positions_for_llm = ''
if positions:
    for position in positions:
        position = position.get('position')
        position_for_llm = f"Current {position.get('coin')} position: size {position.get('szi')} {position.get('coin')}, value of {position.get('positionValue')} usd, leverage {position.get('leverage').get('value')}, and unrealizedPnl {position.get('unrealizedPnl')} usd. The max leverage for this position is {position.get('maxLeverage')}.\n"
        positions_for_llm += position_for_llm
print(positions_for_llm)

Available balance: 324920.281629 USD
Current ATOM position: size 491.45 ATOM, value of 3064.6822 usd, leverage 20, and unrealizedPnl -2327.752925 usd. The max leverage for this position is 50.
Current BTC position: size -0.15792 BTC, value of 14630.18256 usd, leverage 20, and unrealizedPnl -9649.670016 usd. The max leverage for this position is 50.
Current ETH position: size -2.8225 ETH, value of 9526.502 usd, leverage 20, and unrealizedPnl -3857.62365 usd. The max leverage for this position is 50.
Current BNB position: size -14.872 BNB, value of 10540.82744 usd, leverage 20, and unrealizedPnl -5390.202207 usd. The max leverage for this position is 50.
Current GMT position: size 232.0 GMT, value of 36.17576 usd, leverage 2, and unrealizedPnl -38.873456 usd. The max leverage for this position is 50.
Current DYDX position: size -200.7 DYDX, value of 294.32655 usd, leverage 20, and unrealizedPnl 313.53354 usd. The max leverage for this position is 50.
Current APE position: size 2689.3 APE

In [36]:
# [Celda 6] - Información adicional
# Esta parte no cambia
recent_headlines = ("*ROBINHOOD: CRYPTO TRADING VOLUMES OVER $30B, UP 600% YOY\n"
                   "*FED'S MUSALEM: TIME MAY BE APPROACHING TO SLOW OR PAUSE RATE CUTS\n")

chat_summary = "Bob is happy with the Ethereum roadmap and has been hearing more people talk about it.\n"

chat_and_data_summary = chat_summary + recent_headlines

In [37]:
# [Celda 7] - Definición de prompts
# El contenido es similar, pero el formato es específico para Anthropic
system_prompt = (
    "# Instructions:\n"
    "Act as a helpful cryptocurrency assistant helping users manage their trading portfolio.\n"
    "They understand it is inherently risky to trade cryptocurrency, and they want to make sure they are making informed decisions.\n"
    # ... [resto del system_prompt igual que antes] ...
)

user_message = (
    "# Instructions:\n"
    "Here are some details about me, can you help me make decisions about my trading portfolio?\n"
    f"# Risk Level\n{identity_map.get('risk_level')}\n"
    f"# Available Balance\n{available_balance}\n"
    f"# Open Positions\n{positions_for_llm}\n"
    f"# Recent Information\n{chat_and_data_summary}\n"
)

In [38]:
# [Celda 8] - Definición del formato de respuesta
# Esta parte no cambia
from pydantic import BaseModel, Field
from typing import Literal

class Position(BaseModel):
    market: str = Field(..., description="The asset to trade")
    direction: Literal["long", "short"] = Field(..., description="The direction to trade")
    size: float = Field(..., description="The size of the trade denominated in USD. It should be greater than 10.")
    reasoning: list[str] = Field(default_factory=list, description="The reasoning for the decision")
    leverage: int | None = Field(None, description="Optional leverage multiplier")

class PositionReasoning(BaseModel):
    positions_to_maintain: list[Position] = Field(default_factory=list)
    positions_to_modify: list[Position] = Field(default_factory=list)
    positions_to_open: list[Position] = Field(default_factory=list)

In [44]:
# [Celda 9] - Llamada a la API
# Modificamos el mensaje del usuario para solicitar específicamente un formato JSON
user_message_with_format = (
    user_message + "\n\n"
    "Please respond ONLY with a JSON object that follows this exact structure, with no additional text before or after:\n"
    "{\n"
    '  "positions_to_maintain": [],\n'
    '  "positions_to_modify": [],\n'
    '  "positions_to_open": []\n'
    "}\n"
    "Each position should have: market (string), direction ('long' or 'short'), size (float > 10), reasoning (array of strings), and optional leverage (integer)."
)

messages = [
    {
        "role": "user",
        "content": user_message_with_format
    }
]

# DIFERENCIA: Formato específico de Anthropic para la llamada

# Claude 3.5 Sonnet claude-3-5-sonnet-20241022
# Claude 3.5 Haiku claude-3-5-haiku-20241022
# Claude 3 Opus claude-3-opus-20240229
# Claude 3 Sonnet claude-3-sonnet-20240229 OK
# Claude 3 Haiku claude-3-haiku-20240307

completion = anthropic_client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=4096,
    messages=messages,
    system=system_prompt,
    temperature=0.7
)

# DIFERENCIA: Procesamiento de la respuesta de Anthropic
try:
    # Extraemos el texto del TextBlock
    response_text = completion.content[0].text
    
    # Intentamos encontrar y extraer solo la parte JSON
    import re
    json_match = re.search(r'(\{[\s\S]*\})', response_text)
    if json_match:
        json_str = json_match.group(1)
        result = PositionReasoning.model_validate_json(json_str)
    else:
        raise ValueError("No JSON found in response")
        
except Exception as e:
    print(f"Error processing response: {e}")
    print("\nRaw response:")
    print(completion.content)

In [45]:
# [Celda 10] - Visualización de resultados
# Nueva celda para mejor visualización
print("\nPositions to maintain:")
for pos in result.positions_to_maintain:
    print(f"- {pos.market}: {pos.direction} {pos.size} (leverage: {pos.leverage})")
    print(f"  Reasoning: {', '.join(pos.reasoning)}")

print("\nPositions to modify:")
for pos in result.positions_to_modify:
    print(f"- {pos.market}: {pos.direction} {pos.size} (leverage: {pos.leverage})")
    print(f"  Reasoning: {', '.join(pos.reasoning)}")

print("\nPositions to open:")
for pos in result.positions_to_open:
    print(f"- {pos.market}: {pos.direction} {pos.size} (leverage: {pos.leverage})")
    print(f"  Reasoning: {', '.join(pos.reasoning)}")


Positions to maintain:
- ETH: short -2.8225 (leverage: None)
  Reasoning: Despite positive Ethereum roadmap news, maintaining short position until clear trend reversal

Positions to modify:
- BTC: short -0.08 (leverage: 10)
  Reasoning: Reduce exposure due to high unrealized losses, Lower leverage to decrease risk, Fed comments suggest potential monetary policy shift
- APE: long 1000.0 (leverage: 10)
  Reasoning: Current position showing significant losses, Reduce position size and leverage to manage risk

Positions to open:
- ETH: long 15.0 (leverage: 5)
  Reasoning: Positive Ethereum roadmap developments, Increased market interest, Lower leverage for risk management
